In [1]:
import pandas as pd
import numpy as np
import script as sc
import tensorflow as tf

/Users/yashmehra/Library/Python/3.6/lib/python/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
tf.logging.set_verbosity(tf.logging.INFO)

# Data Import

In [5]:
# data = pd.read_csv(r"C:\Users\yashm\Google Drive\Data Capstone_\Project Folder\PreWideData\cnn_wide.csv")
data = pd.read_csv(sc.load_Data('AGG-Yash', 'cnn_wide.csv'))
data.shape

(116090, 243)

In [6]:
data.head()

,time,0,1,2,3,4,5,6,7,8,...,232,233,234,235,236,237,238,239,labels,activity
0,0.0,-0.605,0.617,0.070,-0.621,0.602,0.023,-0.605,0.590,-0.016,...,0.934,-0.133,-0.609,0.918,-0.191,-0.602,0.891,-0.195,0,0
1,1.0,-0.512,0.914,-0.191,-0.441,0.918,-0.152,-0.391,0.914,-0.102,...,0.723,0.613,-0.680,0.727,0.598,-0.680,0.719,0.586,0,0
2,2.0,-0.672,0.711,0.586,-0.648,0.684,0.582,-0.621,0.668,0.574,...,0.359,0.473,-0.867,0.359,0.434,-0.895,0.363,0.422,0,0
3,3.0,-0.898,0.379,0.426,-0.879,0.387,0.441,-0.859,0.402,0.445,...,0.402,0.465,-0.875,0.398,0.445,-0.859,0.391,0.441,0,0
4,4.0,-0.848,0.395,0.441,-0.848,0.379,0.438,-0.848,0.367,0.426,...,0.496,0.418,-0.801,0.496,0.422,-0.801,0.492,0.422,0,0


In [37]:
labels = data['activity']
K = len(data['activity'].unique())
features = data[data.columns[1:-2]]

In [38]:
features.shape, len(labels), K

((116090, 240), 116090, 2)

In [39]:
# This is a sanity check to make sure TensorFlow is reshaping the features 
# into the 3 channels (x, y, z) correctly.
graph = tf.Graph()
with graph.as_default():
    train_features = tf.placeholder(tf.float32, shape=(None, 240))
    x = tf.reshape(train_features, [-1, 80, 3])
    
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print(sess.run(x, feed_dict={train_features: features}))

[[[-0.605  0.617  0.07 ]
  [-0.621  0.602  0.023]
  [-0.605  0.59  -0.016]
  ...
  [-0.543  0.934 -0.133]
  [-0.609  0.918 -0.191]
  [-0.602  0.891 -0.195]]

 [[-0.512  0.914 -0.191]
  [-0.441  0.918 -0.152]
  [-0.391  0.914 -0.102]
  ...
  [-0.688  0.723  0.613]
  [-0.68   0.727  0.598]
  [-0.68   0.719  0.586]]

 [[-0.672  0.711  0.586]
  [-0.648  0.684  0.582]
  [-0.621  0.668  0.574]
  ...
  [-0.84   0.359  0.473]
  [-0.867  0.359  0.434]
  [-0.895  0.363  0.422]]

 ...

 [[ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  ...
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]]

 [[ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  ...
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]]

 [[ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  ...
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]
  [ 0.762  0.258 -0.625]]]


# Hyper Parameter Tuning

In [40]:
num_epochs = 200
learning_rate = 1e-2

In [ ]:
graph = tf.Graph()
with graph.as_default():
    # Placeholders for the actual data.
    train_features = tf.placeholder(tf.float32, shape=(None, 240))
    train_labels = tf.placeholder(tf.int32, shape=(None, ))
    
    # Reshaping the data
    x = tf.reshape(train_features, [-1, 80, 3])
    y = tf.one_hot(
        train_labels,
        depth=K
    )
    
    # Convolution 1
    conv1 = tf.layers.conv1d(inputs=x, 
                             filters=8, 
                             kernel_size=8, 
                             padding="same", 
                             data_format="channels_last",
                             activation=tf.nn.relu)
    # Max Pooling 1 (reduces samples from 80 --> 26)
    pool1 = tf.layers.max_pooling1d(inputs=conv1,
                                    pool_size=5, 
                                    strides=3,
                                    data_format="channels_last")
    
    # Convolution 2
    conv2 = tf.layers.conv1d(inputs=pool1, 
                             filters=16, 
                             kernel_size=8, 
                             padding="same", 
                             data_format="channels_last",
                             activation=tf.nn.relu)
    # Max Pooling 2 (reduces samples from 26 --> 6)
    pool2 = tf.layers.max_pooling1d(inputs=conv2,
                                    pool_size=6, 
                                    strides=4,
                                    data_format="channels_last")
    
    # Flatten the Pooled Data
    pool2_flat = tf.reshape(pool2, [-1, 6 * 16])
    
    # Dense Layer (try adding dropout?)
    dense = tf.layers.dense(inputs=pool2_flat, units=128, activation=tf.nn.relu)
    
    # Logits Layer
    logits = tf.layers.dense(inputs=dense, units=K)
    
    # Define loss function and optimizer
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            logits=logits, 
            labels=y
        )
    )
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # Define accuracy (so that it can be calculated and printed)
    accuracy = tf.reduce_mean(
        tf.cast(
            tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1)),
            tf.float32
        )
    )
    
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    for _ in range(num_epochs):
        feed_dict = {
            train_features: features,
            train_labels: labels
        }
        _, _, acc = sess.run(
            [optimizer, loss, accuracy], 
            feed_dict=feed_dict)
        print(acc)

0.5575243
0.55998796
0.5610992
0.5630459
0.56522524
0.5672582
0.5694203
0.57099664
0.5728831
0.5741838
0.57615644
0.5774658
0.578956
0.5806185
0.5818675
0.5822293
0.58306485
0.58090276
0.58172107
0.5828409
0.5839349
0.58545095
0.5867861
0.5882419
0.58958566
0.590671
0.5918167
0.59327245
0.5951417
0.59658027
0.59782064
0.5989835
0.6002843
0.60145575
0.60304934
0.60422945
0.6055991
0.6070979
0.6085106
0.609708
0.6111724
0.6123956
0.61354125
0.61486775
0.6160565
0.61752087
0.61843395
0.619304
0.6209837
0.62218106
0.6237919
0.6251529
0.62665176
